## Store Voters in Sqlite database
We will store users in sqlite database, getting data from an excel spreadsheet

First connect to the database and insert the ethnicity values in the database before anything else. 

In [19]:
# Store the ethnicities values before populating the database with users:
import sqlite3

db_path = 'db.sqlite3'

queries = [
    "INSERT INTO voteapp_ethnicity (name) VALUES ('black');",
    "INSERT INTO voteapp_ethnicity (name) VALUES ('white');",
    "INSERT INTO voteapp_ethnicity (name) VALUES ('coloured');",
    "INSERT INTO voteapp_ethnicity (name) VALUES ('indian');"
]
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Execute the SQL queries
for query in queries:
    cursor.execute(query)
conn.commit()
conn.close()


Insert gender values

In [ ]:
# Store the genders values before populating the database with users:
import sqlite3

db_path = 'db.sqlite3'

queries = [
    "INSERT INTO voteapp_gender (name) VALUES ('male');",
    "INSERT INTO voteapp_gender (name) VALUES ('female');",
]
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Execute the SQL queries
for query in queries:
    cursor.execute(query)
conn.commit()
conn.close()

In [20]:
import pandas as pd
import openpyxl
from openpyxl import load_workbook

class Voter(object):
    def __init__(self, firstname, lastname, gender, idnumber, ethnicity, province_address,
                 district_address, national_vote, provincial_vote, district_vote):
        self.Firstname = firstname
        self.Lastname = lastname
        self.Gender = gender
        self.Idnumber = idnumber
        self.Ethnicity = ethnicity
        self.Province_address = province_address
        self.District_address = district_address
        self.National_vote = national_vote
        self.Provincial_vote = provincial_vote
        self.District_vote = district_vote

    @classmethod
    def from_excel_row(cls, row):
        """
            Creates a Voter object from a row of data in Excel.
            Returns the voter object
        """
        firstname = row[0]
        lastname = row[1]
        idnumber = row[2]
        ethnicity = row[3]
        province_address = row[4]
        district_address = row[5]
        national_vote = row[6]
        provincial_vote = row[7]
        district_vote = row[8]
        return cls(firstname, lastname, idnumber, ethnicity, province_address,
                   district_address, national_vote, provincial_vote, district_vote)

    @classmethod
    def load_from_excel(cls, file_path):
        """
        Loads voter data from an Excel file.

        Parameters:
        - file_path (str): Path to the Excel file.
        """
        voters = []
        wb = load_workbook(filename= file_path)
        ws = wb.active

        for row in ws.iter_rows(min_row=1, values_only=True):
            voter = cls.from_excel_row(row)
            voters.append(voter)

        return voters
################################# end of class

# # Example usage to load data from Excel file
file_path = 'voter_data.xlsx'
voters = Voter.load_from_excel(file_path)

# print voter data:
for voter in voters[1:3]: ## let it only print the top 2 items
    print(f"Firstname: {voter.Firstname}")
    print(f"Lastname: {voter.Lastname}")
    print(f"ID Number: {voter.Idnumber}")
    print(f"Ethnicity: {voter.Ethnicity}")
    print(f"Province: {voter.Province_address}")
    print(f"District: {voter.District_address}")
    print(f"National Vote: {voter.National_vote}")
    print(f"Provincial Vote: {voter.Provincial_vote}")
    print(f"District Vote: {voter.District_vote}")
    print("=" * 30)  



Firstname: Nancy
Lastname: Long
ID Number: 1703277033901
Ethnicity: white
Province: Western Cape
District: City of Cape Town Metropolitan
National Vote: BOSA
Provincial Vote: COPE
District Vote: BOSA
Firstname: Lori
Lastname: Bright
ID Number: 7603210926186
Ethnicity: indian
Province: Limpopo
District: Waterburg
National Vote: MK
Provincial Vote: ACT
District Vote: EFF


### Storing the voters in the SQLITE database "mydatabase.sqlite"

In [21]:
import sqlite3

# Function to create a connection to the database:
def create_connection(db_file = "db.sqlite3"):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
    return conn

# Get Ethnicity ID:
def get_ethnicity_id(conn, ethnicity_name):
    """Retrieve ethnicity from table, given the ethnicity string """
    cur = conn.cursor()
    cur.execute("SELECT id FROM voteapp_ethnicity WHERE name=?", (ethnicity_name,))
    row = cur.fetchone()
    if row:
        return row[0]
    else:
        return None
    
# Get or create the address:
def get_or_create_address_id(conn, province, district):
    """
    Retrieve the address ID from the Address table based on province and district.
    If the address does not exist, create a new record and return the new ID.
    """
    cur = conn.cursor()
    cur.execute("SELECT id FROM voteapp_address WHERE province=? AND district=?", (province, district))
    row = cur.fetchone()
    if row:
        return row[0]
    else:
        cur.execute("INSERT INTO voteapp_address (province, district) VALUES (?, ?)", (province, district))
        conn.commit()
        return cur.lastrowid

def get_vote_id(conn, national, provincial, regional):
    cur = conn.cursor()
    cur.execute("SELECT id FROM voteapp_vote WHERE national=? AND provincial=? AND regional=?", (national, provincial, regional))
    row = cur.fetchone()
    if row:
        return row[0]
    else:
        cur.execute("INSERT INTO voteapp_vote (national, provincial, regional) VALUES (?, ?, ?)", (national, provincial, regional))
        conn.commit()
        return cur.lastrowid


def insert_voter(conn, voter):
    """ Insert a voter into the Voter table. """
    ethnicity_id = get_ethnicity_id(conn, voter.Ethnicity)
    address_id = get_or_create_address_id(conn, voter.Province_address, voter.District_address)
    vote_id = get_vote_id(conn, voter.National_vote, voter.Provincial_vote, voter.District_vote)

    sql = ''' INSERT INTO voteapp_voter(firstname, lastname, idnumber, ethnicity_id, address_id, vote_id)
              VALUES(?, ?, ?, ?, ?, ?) '''
    cur = conn.cursor()
    cur.execute(sql, (voter.Firstname, voter.Lastname, voter.Idnumber, ethnicity_id, address_id, vote_id))
    conn.commit()
    return cur.lastrowid



Execute the code

In [22]:
# TEST CODE
connection = create_connection()
ethnicityId = get_ethnicity_id(connection, "indian")
addressId = get_or_create_address_id(connection, "Limpopo", "Vhembe")
print(addressId)
print(ethnicityId)

1
4


In [23]:
file_path = 'voter_data.xlsx'
voters = Voter.load_from_excel(file_path)
connection = create_connection()
with connection:
        for voter in voters[1:]:  
            voter_id = insert_voter(connection, voter)